In [35]:
%matplotlib inline
from scipy.stats import randint as sp_randint
from scipy.stats import uniform
from scipy.stats import uniform as sp_randFloat
from sklearn import svm
from tqdm import tqdm
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.naive_bayes import CategoricalNB
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from time import time
from tabulate import tabulate
import numpy as np
import pandas as pd
import sklearn
import warnings
warnings.filterwarnings('ignore')
import os
from scipy.stats import loguniform
from sklearn.linear_model import LogisticRegression

In [36]:
from scipy.stats import randint as sp_randInt

from sklearn.model_selection import GridSearchCV, PredefinedSplit
from sklearn.metrics import make_scorer
from scipy import sparse

In [37]:
import json
with open('GA_output_ET.json', 'r') as fp:
    feature_list = json.load(fp)

In [38]:
feature_list

{'SYN': ['Src Port',
  'Tot Fwd Pkts',
  'Tot Bwd Pkts',
  'TotLen Bwd Pkts',
  'Fwd Pkt Len Max',
  'Bwd Pkt Len Max',
  'Bwd Pkt Len Min',
  'Bwd Pkt Len Mean',
  'Bwd Pkt Len Std',
  'Flow IAT Mean',
  'Flow IAT Std',
  'Flow IAT Max',
  'Flow IAT Min',
  'Fwd IAT Tot',
  'Fwd IAT Mean',
  'Fwd IAT Max',
  'Fwd IAT Min',
  'Bwd IAT Max',
  'Fwd Pkts/s',
  'Bwd Pkts/s',
  'Pkt Len Max',
  'Pkt Len Std',
  'SYN Flag Cnt',
  'PSH Flag Cnt',
  'Pkt Size Avg',
  'Subflow Fwd Pkts',
  'Subflow Fwd Byts',
  'Subflow Bwd Byts',
  'Fwd Act Data Pkts',
  'Active Std',
  'Active Max',
  'Idle Mean',
  'Idle Std',
  'Idle Max',
  'Idle Min',
  'Label'],
 'HTTP': ['Dst Port',
  'Flow Duration',
  'Flow Pkts/s',
  'Flow IAT Max',
  'Fwd IAT Tot',
  'Fwd IAT Min',
  'Fwd Pkts/s',
  'FIN Flag Cnt',
  'SYN Flag Cnt',
  'Label'],
 'ACK': ['TotLen Bwd Pkts',
  'Fwd Pkt Len Max',
  'Fwd Pkt Len Mean',
  'Fwd Pkt Len Std',
  'Flow Byts/s',
  'Flow IAT Std',
  'Fwd IAT Tot',
  'Fwd IAT Mean',
  'Fwd IAT 

In [39]:
file_list={
"SYN":[ '../csvs\\dos-synflooding-1-dec.pcap_Flow.csv', '../csvs\\dos-synflooding-2-dec.pcap_Flow.csv',],
"HTTP":['../csvs\\mirai-httpflooding-4-dec.pcap_Flow.csv', '../csvs\\mirai-httpflooding-1-dec.pcap_Flow.csv'],
"ACK":['../csvs\\mirai-ackflooding-4-dec.pcap_Flow.csv','../csvs\\mirai-ackflooding-1-dec.pcap_Flow.csv'],
"UDP":[ '../csvs\\mirai-udpflooding-4-dec.pcap_Flow.csv','../csvs\\mirai-udpflooding-1-dec.pcap_Flow.csv'],
"ARP":[ '../csvs\\mitm-arpspoofing-6-dec.pcap_Flow.csv', '../csvs\\mitm-arpspoofing-4-dec.pcap_Flow.csv'],
"SP":['../csvs\\scan-hostport-3-dec.pcap_Flow.csv','../csvs\\scan-hostport-4-dec.pcap_Flow.csv'],
"BF":[ '../csvs\\mirai-hostbruteforce-5-dec.pcap_Flow.csv','../csvs\\mirai-hostbruteforce-3-dec.pcap_Flow.csv'],
"OS":['../csvs\\scan-portos-3-dec.pcap_Flow.csv','../csvs\\scan-portos-6-dec.pcap_Flow.csv'],
"SCHD":['../csvs\\scan-hostport-3-dec.pcap_Flow.csv','../csvs\\scan-hostport-4-dec.pcap_Flow.csv'],
"MHDis":['../csvs\\mirai-hostbruteforce-3-dec.pcap_Flow.csv','../csvs\\mirai-hostbruteforce-2-dec.pcap_Flow.csv']}

In [40]:
def run_random_search(model, params, x_train, y_train):
    #grid = GridSearchCV(model, params, cv = ps, n_jobs = -1, scoring = score, verbose = 0, refit = False)
    grid =RandomizedSearchCV(model, param_grid, cv=ps,scoring = 'f1_macro')
    grid.fit(x_train, y_train)
    return (grid.best_params_, round(grid.best_score_,8),grid.best_estimator_)

In [41]:
def find_the_way(path,file_format,con=""):
    files_add = []
    # r=root, d=directories, f = files
    for r, d, f in os.walk(path):
        for file in f:
            if file_format in file:
                if con in file:
                    files_add.append(os.path.join(r, file))  
            
    return files_add

In [42]:
import joblib  # for saving models

# Assume `results` is the DataFrame you are using to collect the training details

# Loop over all attack types and train a model for each
for attack_type in file_list:
    print(f"Training model for {attack_type} attack")

    # Load data for the current attack
    df_train = pd.read_csv(file_list[attack_type][0], usecols=feature_list[attack_type])
    X_train = df_train.iloc[:, 0:-1]
    df_train['Label'] = df_train['Label'].astype('category')
    y_train = df_train['Label'].cat.codes

    df_test = pd.read_csv(file_list[attack_type][1], usecols=feature_list[attack_type])
    X_test = df_test.iloc[:, 0:-1]
    df_test['Label'] = df_test['Label'].astype('category')
    y_test = df_test['Label'].cat.codes

    X = np.concatenate([X_train, X_test])
    test_fold = [-1 for _ in range(X_train.shape[0])] + [0 for _ in range(X_test.shape[0])]
    y = np.concatenate([y_train, y_test])
    ps = PredefinedSplit(test_fold)

    # Hyperparameter tuning and model training
    param_grid = {
        'criterion': ['gini', 'entropy'],
        'max_depth': np.linspace(1, 32, 32, endpoint=True).astype(int),
        'min_samples_split': sp_randint(2, 10),
        'max_features': sp_randint(1, X_train.shape[1])
    }

    second = time()
    f1 = []
    clf = DecisionTreeClassifier()

    # Fit the model and calculate the F1 score
    for ii in range(25):
        clf.fit(X_train, y_train)
        predict = clf.predict(X_test)
        f1.append(sklearn.metrics.f1_score(y_test, predict, average="macro"))
    avg_f1 = sum(f1) / len(f1)

    print(f"Training completed for {attack_type} with average F1: {avg_f1}")

    # Save the trained model for each attack type
    model_filename = f"{attack_type}_model.pkl"
    joblib.dump(clf, model_filename)
    print(f"Model saved as {model_filename}")

# Optionally, save results (F1 scores, hyperparameters, etc.)
results = pd.DataFrame(lines[1:], columns=lines[0])
results.to_csv("DT_HPO.csv", index=False)


Training model for SYN attack
Training completed for SYN with average F1: 0.9856071272565677
Model saved as SYN_model.pkl
Training model for HTTP attack
Training completed for HTTP with average F1: 0.9741554075117895
Model saved as HTTP_model.pkl
Training model for ACK attack
Training completed for ACK with average F1: 0.4620362843647808
Model saved as ACK_model.pkl
Training model for UDP attack
Training completed for UDP with average F1: 0.9665453072119738
Model saved as UDP_model.pkl
Training model for ARP attack
Training completed for ARP with average F1: 0.9593771512254115
Model saved as ARP_model.pkl
Training model for SP attack
Training completed for SP with average F1: 0.9801984737540278
Model saved as SP_model.pkl
Training model for BF attack
Training completed for BF with average F1: 0.7348688085176674
Model saved as BF_model.pkl
Training model for OS attack
Training completed for OS with average F1: 0.9414984014296279
Model saved as OS_model.pkl
Training model for SCHD attack

# RandomizedSearchCV  DT

In [22]:
lines=[['criterion', 'max_depth', 'max_features', 'min_samples_split', "F1","Std","Time","No","Attack"]]

for j in file_list:
    print(j)
    df=pd.read_csv(file_list[j][0],usecols=feature_list[j]) 
    X_train = df.iloc[:,0:-1]
    df['Label'] = df['Label'].astype('category')
    y_train=df['Label'].cat.codes  

    df=pd.read_csv(file_list[j][1],usecols=feature_list[j]) 
    X_test = df.iloc[:,0:-1]
    df['Label'] = df['Label'].astype('category')
    y_test=df['Label'].cat.codes  
       
    X= np.concatenate([X_train, X_test])
    test_fold = [-1 for _ in range(X_train.shape[0])] + [0 for _ in range(X_test.shape[0])]
    y = np.concatenate([y_train, y_test])
    ps = PredefinedSplit(test_fold)
        
    print ('%-35s %-20s %-8s %-8s' % ("HYPERPARAMETERS","F1 Score", "Time", "No"))

    nfolds=10
    param_grid = { 'criterion':['gini','entropy'],
                      "max_depth":np.linspace(1, 32, 32, endpoint=True).astype(int),
                     "min_samples_split": sp_randint(2,10),#uniform(0.1,1 ),
                        # "min_samples_leafs" : np.linspace(0.1, 0.5, 5, endpoint=True)
                        "max_features" : sp_randint(1,X_train.shape[1])}

    second=time()
    f1=[]
    clf=DecisionTreeClassifier()
    for ii in range(25):
        clf.fit(X_train, y_train)
        predict =clf.predict(X_test)
        f1.append(sklearn.metrics.f1_score(y_test, predict,average= "macro") )
    f1=sum(f1)/len(f1)   
    #if f1>0.76:
    print('%-35s %-20s %-8s %-8s' % ("default",f1,round(time()-second,3),ii))
    ######################################################################################################################
    for i in tqdm(range(50)):
        second=time()
        a,b,clf=run_random_search(DecisionTreeClassifier(),param_grid,X,y)
        f1=[]
        for ii in range(10):
            clf.fit(X_train, y_train)
            predict =clf.predict(X_test)
            f1.append(sklearn.metrics.f1_score(y_test, predict,average= "macro") )
        f1_result=sum(f1)/len(f1)   
        f1=np.array(f1)
        stndtd=f1.std()
        temp=list(a.values())
        #print('%-90s %-20s %-8s %-8s' % (a,f1_result,round(time()-second,3),i))
        temp=temp+[f1_result,stndtd,round(time()-second,3),i,j]
        lines.append(temp)

        #if f1>0.76:
results = pd.DataFrame (lines[1:], columns = lines[0])
results.to_csv("DT_HPO.csv",index=False)

final_parametres=[['criterion', 'max_depth', 'max_features', 'min_samples_split', "F1","Std","Time","No","Attack"]]
for i in results["Attack"].unique():
    df=results[results["Attack"]==i]
    m=df["F1"].max()
    df=df[df["F1"]==m]
    m=df["max_depth"].min()
    df=df[df["max_depth"]==m]  
    final_parametres.append(list(df.values)[0])
results = pd.DataFrame (final_parametres[1:], columns=  final_parametres[0])
print (tabulate(results, headers=list(results.columns)))

SYN
HYPERPARAMETERS                     F1 Score             Time     No      
default                             0.9850063894744975   2.183    24      


100%|██████████| 50/50 [01:23<00:00,  1.67s/it]


HTTP
HYPERPARAMETERS                     F1 Score             Time     No      
default                             0.9743787169400595   0.587    24      


100%|██████████| 50/50 [00:15<00:00,  3.27it/s]


ACK
HYPERPARAMETERS                     F1 Score             Time     No      
default                             0.4662332748084681   0.376    24      


...

SCHD
HYPERPARAMETERS                     F1 Score             Time     No      
default                             0.9843500417757266   0.236    24      


100%|██████████| 50/50 [00:10<00:00,  4.85it/s]


MHDis
HYPERPARAMETERS                     F1 Score             Time     No      
default                             0.5092454019748061   0.683    24      


100%|██████████| 50/50 [00:17<00:00,  2.92it/s]

    criterion      max_depth    max_features    min_samples_split        F1          Std    Time    No  Attack
--  -----------  -----------  --------------  -------------------  --------  -----------  ------  ----  --------
 0  gini                  18              22                    4  0.991931  0.00229959    1.384    46  SYN
 1  gini                  28               8                    2  0.974435  0.00316948    0.307    45  HTTP
 2  entropy                1              16                    7  0.901951  0             0.219     2  ACK
 3  gini                  19               1                    6  0.739309  0.238632      0.228    43  UDP
 4  entropy               23              39                    3  0.986008  0.00383517    0.247    45  ARP
 5  gini                   3              19                    9  0.99731   1.11022e-16   0.138    36  SP
 6  entropy                8               3                    2  0.75226   0.0111338     0.277     3  BF
 7  entropy          

In [34]:
from joblib import dump  # Thêm thư viện
import os

# Khởi tạo giá trị tốt nhất ban đầu
f1_best = -1
best_model_path = "best_decision_tree_model.pkl"  # Đường dẫn lưu mô hình

lines=[['criterion', 'max_depth', 'max_features', 'min_samples_split', "F1","Std","Time","No","Attack"]]

for j in file_list:
    print(j)
    df=pd.read_csv(file_list[j][0],usecols=feature_list[j]) 
    X_train = df.iloc[:,0:-1]
    df['Label'] = df['Label'].astype('category')
    y_train=df['Label'].cat.codes  

    df=pd.read_csv(file_list[j][1],usecols=feature_list[j]) 
    X_test = df.iloc[:,0:-1]
    df['Label'] = df['Label'].astype('category')
    y_test=df['Label'].cat.codes  
       
    X= np.concatenate([X_train, X_test])
    test_fold = [-1 for _ in range(X_train.shape[0])] + [0 for _ in range(X_test.shape[0])]
    y = np.concatenate([y_train, y_test])
    ps = PredefinedSplit(test_fold)
        
    print ('%-35s %-20s %-8s %-8s' % ("HYPERPARAMETERS","F1 Score", "Time", "No"))

    nfolds=10
    param_grid = { 'criterion':['gini','entropy'],
                      "max_depth":np.linspace(1, 32, 32, endpoint=True).astype(int),
                     "min_samples_split": sp_randint(2,10),#uniform(0.1,1 ),
                        # "min_samples_leafs" : np.linspace(0.1, 0.5, 5, endpoint=True)
                        "max_features" : sp_randint(1,X_train.shape[1])}

    second=time()
    f1=[]
    clf=DecisionTreeClassifier()
    for ii in range(25):
        clf.fit(X_train, y_train)
        predict =clf.predict(X_test)
        f1.append(sklearn.metrics.f1_score(y_test, predict,average= "macro") )
    f1=sum(f1)/len(f1)   
    #if f1>0.76:
    print('%-35s %-20s %-8s %-8s' % ("default",f1,round(time()-second,3),ii))
    ######################################################################################################################
    # Vòng lặp Random Search
    for i in tqdm(range(50)):
        second = time()
        a, b, clf = run_random_search(DecisionTreeClassifier(), param_grid, X, y)
        f1 = []
        for ii in range(10):
            clf.fit(X_train, y_train)
            predict = clf.predict(X_test)
            f1.append(sklearn.metrics.f1_score(y_test, predict, average="macro"))
        f1_result = sum(f1) / len(f1)
        f1 = np.array(f1)
        stndtd = f1.std()
        temp = list(a.values())
        temp = temp + [f1_result, stndtd, round(time() - second, 3), i, j]
        lines.append(temp)

        # Kiểm tra nếu f1_result tốt hơn mô hình trước đó
        if f1_result > f1_best:
            f1_best = f1_result
            # Lưu mô hình tốt nhất
            print(f"Lưu mô hình tốt nhất với F1: {f1_best}")
            dump(clf, best_model_path)  # Lưu mô hình ra file

SYN
HYPERPARAMETERS                     F1 Score             Time     No      
default                             0.9854291117193597   4.066    24      


  2%|▏         | 1/50 [00:01<01:30,  1.85s/it]

Lưu mô hình tốt nhất với F1: 0.9896013123284602


 14%|█▍        | 7/50 [00:20<02:10,  3.04s/it]

Lưu mô hình tốt nhất với F1: 0.989645633648421


...

Lưu mô hình tốt nhất với F1: 0.9907430861982126


 62%|██████▏   | 31/50 [01:40<01:05,  3.45s/it]

Lưu mô hình tốt nhất với F1: 0.9919245921240887


 98%|█████████▊| 49/50 [02:32<00:03,  3.12s/it]


KeyboardInterrupt: 

# LR

In [33]:
from scipy.stats import loguniform
from sklearn.linear_model import LogisticRegression
lines=[["C",'penalty','Solver',  "F1","Std","Time","No","Attack"]]
for j in file_list:
    print(j)
    
    df=pd.read_csv(file_list[j][0],usecols=feature_list[j]) 
    X_train = df.iloc[:,0:-1]
    df['Label'] = df['Label'].astype('category')
    y_train=df['Label'].cat.codes  


    df=pd.read_csv(file_list[j][1],usecols=feature_list[j]) 
    X_test = df.iloc[:,0:-1]
    df['Label'] = df['Label'].astype('category')
    y_test=df['Label'].cat.codes  
    
    X= np.concatenate([X_train, X_test])
    test_fold = [-1 for _ in range(X_train.shape[0])] + [0 for _ in range(X_test.shape[0])]
    y = np.concatenate([y_train, y_test])
    ps = PredefinedSplit(test_fold)
    
    second=time()

    param_grid = {
    'solver': ['newton-cg', 'lbfgs', 'liblinear'],
    'penalty': ['none', 'l1', 'l2', 'elasticnet'],  # Initially, allow all
    'C': loguniform(1e-5, 100)
}

    # Ensure compatibility between the solver and the penalty
    for solver in param_grid['solver']:
        if solver == 'newton-cg':
            param_grid['penalty'] = ['none', 'l2']  # Only 'none' and 'l2' are valid for 'newton-cg'
        elif solver == 'liblinear':
            param_grid['penalty'] = ['l1', 'l2']  # 'liblinear' supports both 'l1' and 'l2'
        else:
            param_grid['penalty'] = ['l2', 'none']  # Other solvers support 'l2' or 'none'
     
    second=time()
    f1=[]
    clf=LogisticRegression()
    for ii in range(1):
        clf.fit(X_train, y_train)
        predict =clf.predict(X_test)
        f1.append(sklearn.metrics.f1_score(y_test, predict,average= "macro") )
    f1=sum(f1)/len(f1)   
    #if f1>0.76:
    print('%-35s %-20s %-8s %-8s' % ("default",f1,round(time()-second,3),ii))
    ######################################################################################################################
    for i in tqdm(range(10)):
        second = time()
        a, b, clf = run_random_search(LogisticRegression(), param_grid, X, y)
        f1 = []
        for ii in range(5):
            clf.fit(X_train, y_train)
            predict = clf.predict(X_test)
            f1.append(sklearn.metrics.f1_score(y_test, predict, average="macro"))
        f1_result = sum(f1) / len(f1)
        f1 = np.array(f1)
        stndtd = f1.std()
        temp = list(a.values())
        temp = temp + [f1_result, stndtd, round(time() - second, 3), i, j]
        lines.append(temp)

results = pd.DataFrame (lines[1:], columns = lines[0])
results.to_csv("LR_HPO.csv",index=False)
print (tabulate(results, headers=list(results.columns)))
    
final_parametres=[["C",'penalty','Solver',  "F1","Std","Time","No","Attack"]]
for i in results["Attack"].unique():
    df=results[results["Attack"]==i]
    m=df["F1"].max()
    df=df[df["F1"]==m]
    final_parametres.append(list(df.values)[0])
results = pd.DataFrame (final_parametres[1:], columns=  final_parametres[0])
print (tabulate(results, headers=list(results.columns)))

SYN
default                             0.8461994330355928   5.353    0       


100%|██████████| 10/10 [38:05<00:00, 228.56s/it]


HTTP
default                             0.29541737104217114  0.271    0       


100%|██████████| 10/10 [00:46<00:00,  4.68s/it]


ACK
default                             0.29489621966617435  0.164    0       


...

SCHD
default                             0.7150097711030177   0.408    0       


100%|██████████| 10/10 [02:02<00:00, 12.21s/it]


MHDis
default                             0.45720977692237275  0.144    0       


100%|██████████| 10/10 [01:06<00:00,  6.67s/it]

               C  penalty    Solver           F1          Std     Time    No  Attack
--  ------------  ---------  ---------  --------  -----------  -------  ----  --------
 0   0.00564328   l1         liblinear  0.870867  0.00648033   219.032     0  SYN
 1   0.566626     l2         newton-cg  0.858273  1.11022e-16   52.225     1  SYN
 2   0.353918     l1         liblinear  0.913969  0.00185179   378.976     2  SYN
 3   9.00746      l1         liblinear  0.965376  0.00284408   350.407     3  SYN
 4  37.9263       l1         liblinear  0.962956  0.00348264   391.518     4  SYN
 5   0.658253     l2         lbfgs      0.871366  1.11022e-16   28.161     5  SYN
 6  66.426        l1         liblinear  0.964019  0.00576496   222.857     6  SYN
 7   0.00805329   l1         liblinear  0.868279  0.0043162    207.187     7  SYN
 8  83.3599       l1         liblinear  0.960927  0.00423275   399.915     8  SYN
 9   0.085671     l2         lbfgs      0.871257  0             35.293     9  SYN
10   0.3